In [1]:
import warnings
warnings.filterwarnings(action='once')

from keras.applications import VGG16
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import plot_model
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from sklearn.decomposition import PCA
from scipy.spatial import distance
from PIL import Image

from utils import get_image_paths, load_image, get_concatenated_images, plot_results, pdf_results
from configs import *
from run_style_similarity_search import *
from style_stack import StyleStack

Using TensorFlow backend.
/Users/yuanhunglo/anaconda/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yuanhunglo/anaconda/envs/keras/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/yuanhunglo/anaconda/envs/keras/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:5201: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/yuanhunglo/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))
/Users/yuanhunglo/Documents/insight/repos/deep-style-curator/_prototype_style_stack/style_stack.py:93: DeprecationWarning: invalid escape sequence \.
  info = re.search(f'{in

In [4]:
# initialize model
image_paths = get_image_paths('../data/bed')
model = VGG16(weights='imagenet', include_top=True)
layer_name, style_feature_model = get_embedding_model(model, 1)

keeping 54 image_paths to analyze


In [5]:
# generate style embeddings
valid_image_paths, style_embeddings = generate_embeddings(style_feature_model, image_paths)
query_image_idx = int(len(valid_image_paths) * random.random())

analyzing image 0 / 54
finished extracting 54 embeddings for 54 images with 0 failures


In [6]:
# generate style features w/ PCA
style_features = pca(style_embeddings, None)

In [7]:
# find similarity from PCA features based on cosine sim.
closest_image_indices_style = get_closest_indices(query_image_idx, style_features)

In [8]:
# save results
results_image_paths_style = [valid_image_paths[i] for i in closest_image_indices_style]
save_image(results_image_paths_style)

Result image saved at ../output/image_result.png


# Modify StyleStack

In [2]:
# build parameters
image_dir = '../data/chair'
model = VGG16(weights='imagenet', include_top=False)
lib_name = 'raw'
n_results = 5

Instructions for updating:
Colocations handled automatically by placer.


In [23]:
# build, save, re-load, query, and plot
stack = StyleStack.build(image_dir, model)
stack.save(lib_name)
del stack

keeping 106 image_paths to analyze
index time: 889.734 ms


In [9]:
# 
stack = StyleStack.load(lib_name);
query_image_path = '../data/dining_table/902.224.07.jpg'
results = stack.query(query_image_path, None, n_results, write_output=False);

query time: 581.479 ms
[72, 4, 23, 18, 65]


In [10]:
# save results
OUTPUT_DIR = '../output/'
save_image([query_image_path] + results['results_files'], OUTPUT_DIR+'table_chair_4.png')

Result image saved at ../output/image_result.png
